In [27]:
pip install pysolr
pip install spacy==2.2.4

Note: you may need to restart the kernel to use updated packages.


In [3]:
!spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [218]:
import spacy

nlp = spacy.load('en_core_web_sm')
doc = nlp("A 58-year-old African-American woman presents to the ER with episodic pressing/burning anterior chest pain that began two days earlier for the first time in her life. The pain started while she was walking, radiates to the back, and is accompanied by nausea, diaphoresis and mild dyspnea, but is not increased on inspiration. The latest episode of pain ended half an hour prior to her arrival. She is known to have hypertension and obesity. She denies smoking, diabetes, hypercholesterolemia, or a family history of heart disease. She currently takes no medications. Physical examination is normal. The EKG shows nonspecific changes.")
len(doc)
doc

A 58-year-old African-American woman presents to the ER with episodic pressing/burning anterior chest pain that began two days earlier for the first time in her life. The pain started while she was walking, radiates to the back, and is accompanied by nausea, diaphoresis and mild dyspnea, but is not increased on inspiration. The latest episode of pain ended half an hour prior to her arrival. She is known to have hypertension and obesity. She denies smoking, diabetes, hypercholesterolemia, or a family history of heart disease. She currently takes no medications. Physical examination is normal. The EKG shows nonspecific changes.

In [45]:
i = 0
n_grams = []
for i in range(i, len(doc)):
    n_gram = []
    n_gram.append(doc[i].text)
    
    if (i<len(doc)-1):
        n_gram.append(doc[i+1].text)
    if (i<len(doc)-2):
        n_gram.append(doc[i+2].text)    
    
    n_grams.append(n_gram)
len(n_grams)
     

120

In [5]:
import pysolr, os

SOLR_URL = 'http://' + os.environ['SOLR_HOST'] + ':8983/solr/mesh3'

# Create a client instance. The timeout and authentication options are not required.
solr = pysolr.Solr(SOLR_URL, always_commit=False)

# Do a health check.
solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":3,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all"}},\n  "status":"OK"}\n'

In [142]:
def search(n_gram):
    separator = ' '
    text_value = separator.join(n_gram)
    query = '{}:\"{}\"'.format('ConceptName', text_value)
#     print(query)
    results = solr.search(query)
    if (len(results) != 0):
        for r in results:
#             print('Resuts',r)
#             print(r['ConceptName'])
            return (r['ConceptName'], len(n_gram))
    elif len(n_gram)-1>0: 
        return search(n_gram[:len(n_gram)-1])
    else: return ('', len(n_gram)-1)

In [42]:
search(['A', '58-year', '-'])
type(['A', '58-year', '-'])

3
ConceptName:"A 58-year -"
2
ConceptName:"A 58-year"
1
ConceptName:"A"


list

In [255]:
n = 0
word_index = 0
labels = ['O' for i in range(len(doc))]
for n_gram in n_grams:
    if n > 1:
        word_index = word_index + 1
        n= n-1
        continue
    
    concept, n = search(n_gram)
    if (concept):
#         print(n_gram)
        for i in range(n):
            labels[word_index+i] = concept
        print(concept, n, word_index)
    
    word_index = word_index + 1

Life 2 31
Back 2 44
Nausea 2 50
Dyspnea 2 55
Hypertension 1 83
Obesity 2 85
Hypercholesterolemia 3 92
Heart 1 100
Disease 2 101


In [256]:
for label, d in zip(labels, doc):
    print(label, d)

O A
O 58-year
O -
O old
O African
O -
O American
O woman
O presents
O to
O the
O ER
O with
O episodic
O pressing
O /
O burning
O anterior
O chest
O pain
O that
O began
O two
O days
O earlier
O for
O the
O first
O time
O in
O her
Life life
Life .
O The
O pain
O started
O while
O she
O was
O walking
O ,
O radiates
O to
O the
Back back
Back ,
O and
O is
O accompanied
O by
Nausea nausea
Nausea ,
O diaphoresis
O and
O mild
Dyspnea dyspnea
Dyspnea ,
O but
O is
O not
O increased
O on
O inspiration
O .
O The
O latest
O episode
O of
O pain
O ended
O half
O an
O hour
O prior
O to
O her
O arrival
O .
O She
O is
O known
O to
O have
Hypertension hypertension
O and
Obesity obesity
Obesity .
O She
O denies
O smoking
O ,
O diabetes
Hypercholesterolemia ,
Hypercholesterolemia hypercholesterolemia
Hypercholesterolemia ,
O or
O a
O family
O history
O of
Heart heart
Disease disease
Disease .
O She
O currently
O takes
O no
O medications
O .
O Physical
O examination
O is
O normal
O .
O The
O EKG
O shows
O n